In [2]:
import os
import certifi
from river import datasets
from river import metrics
from river import tree
from river import ensemble
from river import evaluate
from river import compose
from river import naive_bayes
from time import time

from river import anomaly
from river import compose
from river import datasets
from river import metrics
from river import preprocessing
import pickle
import ray
import pandas as pd
from pathlib import Path 


In [ ]:
@ray.remote
def f(topic,max_cnt_per_consumer,no_of_consumers,consumer_no,total_no_of_partitions,bts,u,p):
    
    max_cnt=max_cnt_per_consumer
    import json
    import sys
    import os
    from json import dumps, loads    
    from confluent_kafka.cimpl import Producer, Consumer, KafkaException, KafkaError

    from datetime import datetime
    import time
    from confluent_kafka import TopicPartition
    from river import datasets
    from river import metrics
    from river import tree
    from river import ensemble
    from river import evaluate
    from river import compose
    from river import naive_bayes
    from time import time

    from river import anomaly
    from river import compose
    from river import datasets
    from river import metrics
    from river import preprocessing
    import pickle
    import statistics
    import certifi
    tls = []
    partitions_per_consumer = int(total_no_of_partitions/no_of_consumers)
    for t in range(partitions_per_consumer):        
        tls.insert(t, TopicPartition(topic, consumer_no*partitions_per_consumer+t))
    group_id=f"group-{no_of_consumers}"
    
    conf = {'bootstrap.servers': bts,
                'sasl.mechanism': 'PLAIN',
                'security.protocol': 'SASL_SSL',
                'sasl.username': u,
                'sasl.password': p,
                'ssl.ca.location': certifi.where(),
                'group.id': group_id,
                'auto.offset.reset': 'latest'}
    consumer = Consumer(conf)
    consumer.assign(tls)
    auc = metrics.ROCAUC()
    f1 = metrics.F1()
    recall = metrics.MicroRecall()
    durs=[]
    i=0
    ignored=0
    mem=[]
    model = None
    try:
        if topic=='HoeffdingAdaptiveTreeClassifier':
            model = tree.HoeffdingAdaptiveTreeClassifier(grace_period=100,  delta=1e-5, leaf_prediction='nb', nb_threshold=10,seed=0)
        elif topic=='SRPClassifierHAT':
            model =  ensemble.SRPClassifier(
                                               model=tree.HoeffdingAdaptiveTreeClassifier(grace_period=100,  delta=1e-5, leaf_prediction='nb', nb_threshold=10, seed=0), seed=42,
                                            )
        elif topic=='SRPClassifierNaiveBayes':
            model = ensemble.SRPClassifier(
                                             model=naive_bayes.BernoulliNB(alpha=1), seed=42,
                                          )
        elif topic=='AdaptiveRandomForestClassifier':
            model = ensemble.AdaptiveRandomForestClassifier(leaf_prediction="mc")
                                          
        elif topic=='HalfSpaceTrees':
            model = compose.Pipeline(preprocessing.MinMaxScaler(),anomaly.HalfSpaceTrees(seed=42))
            
        while(i<max_cnt):
            msg = consumer.poll(timeout=0.1)
            if msg is None: continue
            if msg.error():
                if msg.error().code() == KafkaError._PARTITION_EOF:
                    # End of partition event
                    sys.stderr.write('%% %s [%d] reached end at offset %d\n' %
                                         (msg.topic(), msg.partition(), msg.offset()))
            else:                
                message = loads(msg.value().decode("utf-8"))
                st = message['st']
                data = message['f']
                y = (message['y']=='true')              
               
                try:                    
                    score = model.predict_one(data)
                    model = model.learn_one(data,y)                    
                    end = time()
                    auc = auc.update(y, score)
                    f1 = f1.update(y, score)
                    recall = recall.update(y, score)
                    durs.append(end-st)
                    i = i + 1                
                    if i%1000==0: #Sample model memory every 1000 records
                        mem.append(model._raw_memory_usage)
                except:
                    ignored = ignored + 1
    finally:
        # Close down consumer to commit final offsets.
        consumer.close()    
    mean = statistics.mean(durs)
    median = statistics.median(durs)
    max1 = max(durs)
    min2 = min(durs)
    total_records = len(durs)
    memory_usage = statistics.mean(mem)
    return {'index':consumer_no,'ignored':ignored, 'max':max1,'min':min2,'avg':mean,'median':median,'length':total_records,'auc':auc,'f1':f1,'recall':recall,'avg_model_mem':memory_usage}



In [ ]:
import ray
import os
import pandas as pd
from pathlib import Path  


In [ ]:
print(ray.is_initialized() )
if ray.is_initialized():
    ray.shutdown()
if ray.is_initialized() == False:
    service_host = os.environ["RAY_HEAD_SERVICE_HOST"]
    service_port = os.environ["RAY_HEAD_SERVICE_PORT"]
    ray.init(f"ray://{service_host}:{service_port}")


In [ ]:
#Replace topic and no_of_consumers as needed
max_records = 100000
topic='HoeffdingAdaptiveTreeClassifier'
no_of_partitions = 16
no_of_consumers = 8
max_cnt_per_consumer=int(max_records/no_of_consumers)


b = os.environ['kafka_bootstrap_servers']
u = os.environ['kafka_username']
p = os.environ['kafka_password']
futures = [f.remote(topic,max_cnt_per_consumer,no_of_consumers,consumer_no,no_of_partitions,b,u,p) for consumer_no in range(no_of_consumers)]
results = ray.get(futures)
print(results)
#Start the producer from the file confluence_producer_malicious_url.py

In [ ]:
df= pd.DataFrame.from_records(results)
filepath = Path(f'/mnt/code/results/metrics-{topic}-{no_of_consumers}.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_csv(filepath)  
ray.shutdown()

In [ ]:
@ray.remote
def f_cc(topic,max_cnt_per_consumer,no_of_consumers,consumer_no,total_no_of_partitions,bts,u,p):
    
    max_cnt=max_cnt_per_consumer
    import json
    import sys
    import os
    from json import dumps, loads    
    from confluent_kafka.cimpl import Producer, Consumer, KafkaException, KafkaError

    from datetime import datetime
    import time
    from confluent_kafka import TopicPartition
    from river import datasets
    from river import metrics
    from river import tree
    from river import ensemble
    from river import evaluate
    from river import compose
    from river import naive_bayes
    from time import time

    from river import anomaly
    from river import compose
    from river import datasets
    from river import metrics
    from river import preprocessing
    import pickle
    import statistics
    import certifi
    tls = []
    partitions_per_consumer = int(total_no_of_partitions/no_of_consumers)
    for t in range(partitions_per_consumer):        
        tls.insert(t, TopicPartition(topic, consumer_no*partitions_per_consumer+t))
    group_id=f"group-{no_of_consumers}"
    
    conf = {'bootstrap.servers': bts,
                'sasl.mechanism': 'PLAIN',
                'security.protocol': 'SASL_SSL',
                'sasl.username': u,
                'sasl.password': p,
                'ssl.ca.location': certifi.where(),
                'group.id': group_id,
                'auto.offset.reset': 'latest'}
    consumer = Consumer(conf)
    consumer.assign(tls)
    auc = metrics.ROCAUC()
    f1 = metrics.F1()
    recall = metrics.MicroRecall()
    durs=[]
    i=0
    ignored=0
    mem=[]
    model = None
    try:
        if topic=='HalfSpaceTrees':
            model = compose.Pipeline(preprocessing.MinMaxScaler(),anomaly.HalfSpaceTrees(seed=42))
            
        while(i<max_cnt):
            
            msg = consumer.poll(timeout=0.1)
            if msg is None: continue
            if msg.error():
                if msg.error().code() == KafkaError._PARTITION_EOF:
                    # End of partition event
                    sys.stderr.write('%% %s [%d] reached end at offset %d\n' %
                                         (msg.topic(), msg.partition(), msg.offset()))
            else:                
                message = loads(msg.value().decode("utf-8"))
                st = message['st']
                data = message['f']
    
                y = (message['y']==1)        
                try:                    
                    score = model.score_one(data) 
                    model = model.learn_one(data)

                    end = time()   
                    auc = auc.update(y, score==1)
                    f1 = f1.update(y, score==1)
                    recall = recall.update(y, score==1)
                    durs.append(end-st)
                    i = i + 1                
                    if i%1000==0: #Sample model memory every 1000 records
                        mem.append(model._raw_memory_usage)
                except:
                    ignored = ignored + 1
    except:
        print('exception')
    finally:
        # Close down consumer to commit final offsets.
        consumer.close()    
    mean = statistics.mean(durs)
    median = statistics.median(durs)
    max1 = max(durs)
    min2 = min(durs)
    total_records = len(durs)
    memory_usage = statistics.mean(mem)
    return {'index':consumer_no,'ignored':ignored, 'max':max1,'min':min2,'avg':mean,'median':median,'length':total_records,'auc':auc,'f1':f1,'recall':recall,'avg_model_mem':memory_usage}



In [37]:
print(ray.is_initialized() )
if ray.is_initialized():
    ray.shutdown()
if ray.is_initialized() == False:
    service_host = os.environ["RAY_HEAD_SERVICE_HOST"]
    service_port = os.environ["RAY_HEAD_SERVICE_PORT"]
    ray.init(f"ray://{service_host}:{service_port}")


False


In [38]:
#Replace topic and no_of_consumers as needed
max_records = 100000
topic='HalfSpaceTrees'
no_of_partitions = 16
no_of_consumers = 8
max_cnt_per_consumer=int(max_records/no_of_consumers)


b = os.environ['kafka_bootstrap_servers']
u = os.environ['kafka_username']
p = os.environ['kafka_password']
futures = [f_cc.remote(topic,max_cnt_per_consumer,no_of_consumers,consumer_no,no_of_partitions,b,u,p) for consumer_no in range(no_of_consumers)]
results = ray.get(futures)
print(results)
#Start the producer from the file confluence_producer_credit)card.py

[{'index': 0, 'ignored': 0, 'max': 0.7428181171417236, 'min': 0.047324419021606445, 'avg': 0.08125932992935181, 'median': 0.08186662197113037, 'length': 12500, 'auc': ROCAUC: 50.00%, 'f1': F1: 0.00%, 'recall': MicroRecall: 99.77%, 'avg_model_mem': 1003542.3333333334}, {'index': 1, 'ignored': 0, 'max': 0.7220723628997803, 'min': 0.047197580337524414, 'avg': 0.08225127578735351, 'median': 0.08317673206329346, 'length': 12500, 'auc': ROCAUC: 50.00%, 'f1': F1: 0.00%, 'recall': MicroRecall: 99.78%, 'avg_model_mem': 1003477}, {'index': 2, 'ignored': 0, 'max': 0.7327229976654053, 'min': 0.04799628257751465, 'avg': 0.08202138759613037, 'median': 0.08086156845092773, 'length': 12500, 'auc': ROCAUC: 50.00%, 'f1': F1: 0.00%, 'recall': MicroRecall: 99.77%, 'avg_model_mem': 1003684.6666666666}, {'index': 3, 'ignored': 0, 'max': 0.7649078369140625, 'min': 0.047894954681396484, 'avg': 0.08434619976043702, 'median': 0.08265852928161621, 'length': 12500, 'auc': ROCAUC: 50.00%, 'f1': F1: 0.00%, 'recall'

In [39]:
df= pd.DataFrame.from_records(results)
filepath = Path(f'/mnt/code/results/metrics-{topic}-{no_of_consumers}.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_csv(filepath)  
ray.shutdown()

In [36]:
#If fully automated run this after filling out he missing lines. This cell is intended to give
#an idea of all the experiments that are run for the malicious url use-case
max_records=100000
b = os.environ['kafka_bootstrap_servers']
u = os.environ['kafka_username']
p = os.environ['kafka_password']
for topic in ['HoeffdingAdaptiveTreeClassifier','SRPClassifierHAT','SRPClassifierNaiveBayes','AdaptiveRandomForestClassifier','HalfSpaceTrees']:
    for no_of_consumers in [8,4,2]:
        #Create Topic here
        #Start producing to the topic here in a separate thread
        
        if ray.is_initialized():
            ray.shutdown()
        if ray.is_initialized() == False:
            service_host = os.environ["RAY_HEAD_SERVICE_HOST"]
            service_port = os.environ["RAY_HEAD_SERVICE_PORT"]
            ray.init(f"ray://{service_host}:{service_port}")
        max_cnt_per_consumer=100000/no_of_consumers
        print('Processing Topic'
        futures = [f.remote(topic,max_cnt_per_consumer,no_of_consumers,consumer_no,b,u,p) for consumer_no in range(no_of_consumers)]
        results = ray.get(futures)
        df= pd.DataFrame.from_records(results)
        filepath = Path(f'/mnt/code/results/metrics-{topic}-{no_of_consumers}.csv')  
        filepath.parent.mkdir(parents=True, exist_ok=True)  
        df.to_csv(filepath)  
        #Delete topic
        


SyntaxError: invalid syntax (3138289115.py, line 20)